
<a target="_blank" href="https://colab.research.google.com/github/Rchatru/ML4DS/blob/master/assignment_F.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  </a>

## Import libraries

In [ ]:
import sys

# Detect if running in Google Colab
in_colab = 'google.colab' in sys.modules

if in_colab:
    print("Running in Google Colab. Installing some required libraries...")
    !pip install category_encoders wandb shap tsfresh pyts sktime[all_extras] evalml featuretools imbalanced-learn pyarrow

In [ ]:
# Basic Libraries & data manipulation
import os
from itertools import product
import numpy as np
import pandas as pd
import statistics

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Feature Encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import category_encoders as ce # pip install category_encoders

# Feature Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer

# Feature Engineering
from imblearn.over_sampling import SMOTE

# Modelling & Evaluation
from sklearn.model_selection import train_test_split, StratifiedGroupKFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc


## Not important for now. Could be useful at the model training stage
# import wandb
# import shap
## For installing them in Colab do:
## pip install wandb
## pip install shap

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import the data


#### Option 1- If working in Google Colab:

1. Add the following files to your drive:
https://drive.google.com/file/d/1gKJmVCODGcpCfgVtOgUzlxH1Q1Vf0KaR/view?usp=sharing
https://drive.google.com/file/d/1nydN6iEc-WEn1ZcOw3K1vjEonAuqWWOh/view?usp=sharing
2. In the drive interface, create a shortcut for both files
3. Uncomment the cell and mount your drive using the code below

*Not necessary, now the script detects if running in Colab and runs the required config steps*

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#df = pd.read_csv('/content/drive/MyDrive/train.csv')

#### Option 2- Import the data locally from Github

In [ ]:
#filename = 'train.csv'
#df = pd.read_csv(os.path.join(os.getcwd(),filename))

In [ ]:
if in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    df = pd.read_csv('/content/drive/MyDrive/train.csv')
    comp_df = pd.read_csv('/content/drive/MyDrive/competition.csv')
else:
    filename = 'train.csv'
    filenamecomp = 'competition.csv'
    df = pd.read_csv(os.path.join(os.getcwd(),filename))
    comp_df = pd.read_csv(os.path.join(os.getcwd(),filenamecomp))

## EDA. Exploratory Data Analysis

### Basic Info

We have 329975 samples and 6 features. The features are: `Year`, `Month`, `Consumer_type`, `Consumption`, `Consumer_number` and `Installation_zone`.

The variables `Year`, `Month` and `Consumption` are numerical (type `int64`), while `Consumer_type`, `Consumer_number` and `Installation_zone` are textual (type `object`).

In this case the *target* variable is `Consumer_type`, which is a categorical variable with 7 possible values: `['domestic', 'industrial', 'rural commercial', 'construction', 'low income families', 'rural domestic', 'rural expansion']`. The `domestic`, `industrial` and `low income families` are self-explanatory. The rural* categories are related to agro-production consumers in varying sizes:
- rural domestic: small/familiar companies
- rural commercial: mid-sized companies
- rural expansion: others agro-production

*Note that a priori, the agro-production user's water consumption could be very season dependant (because of the crops cycles and the weather), so it could be interesting to explore this in the future.*

There are no NaN values in the dataset. But in this case, it is important to note the existence of the consumption value `0` in the feature `Consumption`. This could represent a missing value or actually, a zero water usage for that particular consumer in a specific month.

We have 7 years of data, from 2013 to 2020 (except 2015), All the years are complete, with 12 months each.

- Years available:  **[2013, 2014, 2016, 2017, 2018, 2019, 2020]**
- Months:  **[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]**
- Consumer types:  **['domestic', 'industrial', 'rural commercial', 'construction', 'low income families', 'rural domestic', 'rural expansion']**
- Number of Installation zones: **49**, values from **Installation_zone 1 to Installation_zone 49**
- Number of unique Consumers:  **27632**
- Consumption values ranging from **0 to 4978 m3**
- Number of water consumption values equal 0 m3: **61658**


In [ ]:
print(df.head(), '\n')
print(df['Consumer_type'].value_counts())

In [ ]:
# Dataset basic info
display(df.info())
print('Years available: ', df.Year.unique().tolist())
print('Months: ', df.Month.unique().tolist())
print('Number of months available each year: ', df.groupby('Year')['Month'].nunique().tolist())
print('Consumer types: ', df.Consumer_type.unique().tolist())
print(f'Number of Installation zones: {df.Installation_zone.nunique()}, values from {df.Installation_zone.unique()[0]} to {df.Installation_zone.unique()[-1]}')
print('Number of unique Consumers: ', df.Consumer_number.nunique())
print(f'Consumption values ranging from {df.Consumption.min()} to {df.Consumption.max()} m3')
print(f'Number of water consumption values equal 0 m3: {df[df.Consumption == 0].shape[0]}')


### Percentage of 0 m3 Consumption value for each year, month and Consumer type

- Analyzing the proportion of zero values for each year, we see small diferences. The values are almost equally distributed between the years. The highest percentage is for the years 2018, 2020 (~ 15 %), and the lowest for the year 2013, 2017, 2016 (~ 13 %). *Note that the percentages are relative to the total number of zero consumption values. Eg. out of the 61658 existing zero values, 15.19 % corresponds to the year 2018. If we calculate the percentages relative to the total number of samples, the values are much lower, from 2.5 to 2.8 %.*

- If we repeat the same for each month, we can see more differences. The months with the highest percentage of zero values are January, February and December (10.09, 9.18 and 9.99 % respectively). On the other hand, the lowest percentages are for June, July and August (7.59, 7.22 and 6.06 % respectively). *Is there a season trend?*

- Finally, if we analyze by Consumer type, it is considerable the great amount of zero values for the `rural expansion` category (41 %). Also `construction` (35 %) has a high percentage of zero values. On the other end, there are the `domestic` and `low income families` categories (16.30 and 6.60 % respectively). *Note that in this case, the percentages are relative to the total number of samples for each Consumer type. Eg. out of the 890 rural expansion samples, 41,34 % (368) of them corresponds to zero consumption values.*

In [ ]:
print('Proportion of 0 m3 Consumption value by year [%]: \n', df[df.Consumption == 0].value_counts(['Year'], normalize=True)*100, sep='', end='\n\n')
print('Proportion of 0 m3 Consumption value by month [%]: \n', df[df.Consumption == 0].value_counts(['Month'], normalize=True)*100, sep='', end='\n\n')
print('Proportion of 0 m3 Consumption value by Consumer_type [%]: \n', df[df.Consumption == 0].groupby('Consumer_type')['Consumer_number'].count()/df.groupby('Consumer_type')['Consumer_number'].count()*100, sep='', end='\n\n')

# Barplot of the above data as percentages
fig, ax = plt.subplots(1,3, figsize=(15,5))
(df[df.Consumption == 0].groupby('Year')['Consumer_number'].count()/len(df[df.Consumption == 0])*100).plot(kind='bar', ax=ax[0])
(df[df.Consumption == 0].groupby('Month')['Consumer_number'].count()/len(df[df.Consumption == 0])*100).plot(kind='bar', ax=ax[1])
(df[df.Consumption == 0].groupby('Consumer_type')['Consumer_number'].count()/df.groupby('Consumer_type')['Consumer_number'].count()*100).plot(kind='bar', ax=ax[2])
ax[0].set_title('Proportion by Year')
ax[1].set_title('Proportion by Month')
ax[2].set_title('Proportion by Consumer_type')
ax[0].set_ylabel('Percentage')
ax[1].set_ylabel('Percentage')
ax[2].set_ylabel('Percentage')
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=45)
fig.suptitle('Proportion of 0 m3 Consumption value by Year, Month and Consumer_type', fontsize=14)
plt.tight_layout()
plt.show()


#### Percentage of 0 m3 Consumption value for each month and Consumer type

In order to see if there is a different monthly behaviour for each Consumer type, we can plot the percentage of zero values for each month and Consumer type. First, compute the number of zero values by consumer type and month, and then normalize it by each consumer type.

In [ ]:
percent_type_month = df[df.Consumption == 0].groupby(['Consumer_type','Month'])['Consumer_number'].count()/df[df.Consumption == 0].groupby('Consumer_type')['Consumer_number'].count()*100
df_type_month = percent_type_month.unstack().T
display(df_type_month)

(*In order to see the Plotly interactive plots, download and run the notebook in your local machine or in Google Colab. Attached screenshot to visualise the graph without running the notebook.*)

![Percentage of 0 m3 Consumption value for each month and Consumer type](zero-consumption-month-consumer-type.png)

- Interactively filtering the graph traces by consumer, it is clearly visible that the three rural categories have a similar behaviour, with a peak in the months of January and December, and a minimum in the months of June, July and August. Also, the `construction` category shares a comparable trend with them, but with a higher peak in June.

- The `domestic` and `industrial` show a similar and more stable behaviour through the year. `rural domestic` and `rural comercial` also have a similar trend to them until August, when they diverge.

- The `low income families` category has a different behaviour from the rest.


In [ ]:
total_counts = df.groupby(['Consumer_type'])['Consumer_number'].count()

fig = go.Figure()

for consumer_type in df['Consumer_type'].unique():
    
    filtered = df_type_month[consumer_type]

    fig.add_trace(go.Scatter(
        mode='lines+markers',
        x=filtered.index,
        y=filtered.values,
        name=f'{consumer_type}',
    ))

fig.update_layout(

    title='Proportion of 0 m3 Consumption value by Month and Consumer Type',
    xaxis_title='Month',
    yaxis_title='Percentage of 0 m3 Consumption',
    xaxis={'ticktext': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], 
           'tickvals': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
)

fig.show()

### Creating a complete dataframe with all the possible combinations of year, month and consumer_number

When we have missing data for a particular month and consumer, we actually doesn't have the entire sample (row), so we can't just replace the NaN (because they doesn't exist). So the idea is to create a complete dataframe with all the possible combinations of year, month and consumer_number, and then merge it with the original dataframe. This way, we will have a complete dataframe with all the months for each consumer_number, and the missing values will be NaN. Then we can do whatever we want with them.

In the following table we can see that for each consumer there are several months missing. For example, the consumer ``AABK96307399687530``, has missing values for the months 4,5,6,7. It is important to note that in this table when we have a NaN value, it means that we don't have any data for that particular month (throughout the years).

In [ ]:
print(df.groupby(['Consumer_number','Month'])['Consumption'].mean().unstack())
# print(df.groupby(['Consumer_number','Month'])['Consumption'].mean().unstack().isna().sum(axis=0).value_counts())

Average consumption values for each consumer type and month.

In [ ]:
df_mean = df.groupby(['Consumer_type','Month'])['Consumption'].mean().unstack()
display(df_mean)

Itertools.product() is used to create all possible combinations of the 3 variables (year, month and Consumer type). Now, for each consumer, we have the rows for all the months, although some of them will be NaN. Then we have to decide what to do with them.

In [ ]:
# Creating a complete DataFrame with all possible combinations of Year, Month, Consumer_type, and Consumer_number
years = df['Year'].unique()
months = df['Month'].unique()
consumers = df['Consumer_number'].unique()

# Product of all combinations
all_combinations = product(years, months, consumers)
# Convert to DataFrame
complete_data = pd.DataFrame(all_combinations, columns=['Year', 'Month', 'Consumer_number'])

#-------------------- Apply the same process to the competition dataset ---------------------
years = comp_df['Year'].unique()
months = comp_df['Month'].unique()
consumers = comp_df['Consumer_number'].unique()

# Product of all combinations
all_combinations_comp = product(years, months, consumers)
# Convert to DataFrame
complete_data_comp = pd.DataFrame(all_combinations_comp, columns=['Year', 'Month', 'Consumer_number'])

Now we merge the new bigger dataframe with the old one to conserve the the other features (Installation_zone and Consumer_type).

In [ ]:
# Combine the complete DataFrame with the original DataFrame
combined_data = pd.merge(complete_data, df, 
                         on=['Year', 'Month', 'Consumer_number'], 
                         how='left')

print('Shape of the complete DataFrame: ', combined_data.shape)
display(combined_data.head())

#----------------------- Apply the same process to the competition dataset ---------------------
combined_data_comp = pd.merge(complete_data_comp, comp_df, 
                         on=['Year', 'Month', 'Consumer_number'], 
                         how='left')

We could filter by a specific Consumer and notice that now we have all of the months present.

In [ ]:
combined_data.loc[combined_data.Consumer_number == "AABK96307399687530"]

Now we have to complete for each user the columns ``Consumer_type`` and ``Installation_zone``, with the data that we have for other months.

In [ ]:
# creating a dictionary with the mapping values
consumer_type_mapping = df.drop_duplicates('Consumer_number').set_index('Consumer_number')['Consumer_type'].to_dict()
installation_zone_mapping = df.drop_duplicates('Consumer_number').set_index('Consumer_number')['Installation_zone'].to_dict()

# mapping the values
combined_data['Consumer_type'] = combined_data['Consumer_number'].map(consumer_type_mapping)
combined_data['Installation_zone'] = combined_data['Consumer_number'].map(installation_zone_mapping)

combined_data.head()

#----------------------- Apply the same process to the competition dataset ---------------------
installation_zone_mapping_comp = comp_df.drop_duplicates('Consumer_number').set_index('Consumer_number')['Installation_zone'].to_dict()

# mapping the values
combined_data_comp['Installation_zone'] = combined_data_comp['Consumer_number'].map(installation_zone_mapping_comp)

Finally the dataset is complete, and now we have to deal with the NaN values.

In [ ]:
combined_data.loc[combined_data.Consumer_number == "AABK96307399687530"]

In [ ]:
df = combined_data.copy()
# Add Date column for easier manipulation
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(Day=1))
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

df.head()

#----------------------- Apply the same process to the competition dataset ---------------------
df_comp = combined_data_comp.copy()
# Add Date column for easier manipulation
df_comp['Date'] = pd.to_datetime(df_comp[['Year', 'Month']].assign(Day=1))
df_comp['Date'] = df_comp['Date'].dt.strftime('%Y-%m-%d')

### Stats by Installation zone

There are 49 Installation zones, but some of the Consumer types categories are concentrated in a few of them. In fact, there are zones where there is only one type of consumer (see heatmap, the numbers represent percentages for each consumer type). This feature could be useful for the classification task.

- Broadly, the ``construction``, ``domestic``, ``industrial`` and ``low income families`` are located in the Installation zones 1 to 4 (with different distribution over them).

- The rural comercial, rural domestic and rural expansion are more evenly distributed over the Installation zones, (specially rural domestic and rural expansion).

In [ ]:
# For each Consumer_type, the Installation_zone where the are the most Consumers
print(df.groupby('Consumer_type')['Installation_zone'].value_counts().groupby(level=0).head(), end='\n\n')

# For each installation zone, the Consumer_type with the highest number of Consumers
print(df.groupby('Installation_zone')['Consumer_type'].value_counts())


In [ ]:
cross = df.groupby(['Installation_zone', 'Consumer_type'])['Consumer_number'].count().unstack()
cross.replace(np.nan, 0, inplace=True)

# Normalize for each Consumer_type
freqs = cross/cross.sum(axis=0)*100
freqs = freqs.round(2)

# Reorder the index Installation_zone by ascending number of zone, eg. Installation_zone 1, Installation_zone 2, etc.
freqs = freqs.reindex(sorted(freqs.index, key=lambda x: int(x.split()[1])))
print(freqs)


In [ ]:
freqs.sum(axis=1).plot(kind='bar', figsize=(15,5))
plt.title('Number of Consumers per Installation_zone')
plt.ylabel('Number of Consumers')
print('Number of Consumers per Installation_zone: \n', freqs.sum(axis=1), sep='', end='\n\n')

In [ ]:
fig = plt.figure(figsize=(12, 10))
sns.heatmap(freqs, annot=True, cmap="YlGnBu", fmt='g')
plt.title("Heatmap of Consumer Types by Installation Zone")
# Rotate tick marks for visibility
plt.xticks(rotation=30)
plt.show()

In [ ]:
df.loc[df['Consumption']!=0]

#### Histogram of Consumption values by Consumer type

There are some outliers in the consumption values, specially for the ``industrial`` category. There are also some extreme values in the ``domestic``, ``construction`` and ``rural domestic`` categories. This could represent a measurement error or a real extreme value (eg. a big company with a high water consumption, refilling a pool, etc.). In any case, it is important to note that these outliers could affect the performance of the classification model. Also, the values are not normally distributed.

In [ ]:
# Histogram of Consumption values for each Consumer_type (excluding 0 values and before data cleaning)
    
fig = px.histogram(df.loc[df['Consumption']!=0], x='Consumption', nbins=3000, color='Consumer_type', barmode='overlay', histnorm='percent', title='Consumption distribution for each Consumer_type')
fig.update_layout(
    xaxis_title='Consumption [m3]',
    yaxis_title='Percentage (by category) [%]',
)
fig.show()

#### Boxplot of Consumption values by Consumer type

Very high consumption values visible. Zoom in the graph to see the boxplots better.

In [ ]:
fig = px.box(df.loc[df['Consumption']!=0], y='Consumption', color='Consumer_type', title='Consumption distribution for each Consumer_type')
fig.show()

# Violin plot. Similar to boxplot but shows the distribution of the data
# fig = px.violin(df.loc[(df['Consumption']!=0) & (df['Consumption']<600)], y='Consumption', color='Consumer_type', title='Consumption distribution for each Consumer_type')
# fig.show()

## Data Preprocessing

### Data Cleaning

It is necessary to clean the data in order to prepare it for the classification task.

#### 1. Identify and process the missing values (remove or impute)

We have to decide what to do with the NaN values. We could set a threshold and if there are more than X months missing for a particular consumer, we could remove it. If there is only missing data for a few months, we could impute them with the mean of the mean values for that month and consumer_type category that the user belongs to, and the year mean of the consumer, so that we get a balanced values and we don't change too much the distribution of the data.

In this case, it is important to note the existence of the consumption value `0` in the feature `Consumption`. This could represent a missing value or actually, a zero water usage for that particular consumer in a specific month.

#### 2. Remove the rows with negative consumption values

There are no negative values in the dataset.

#### 3. Remove the outliers

There are some outliers in the consumption values, specially for the ``industrial`` category. There are also some extreme values in the ``domestic``, ``construction`` and ``rural domestic`` categories. This could represent a measurement error or a real extreme value (eg. a big company with a high water consumption, refilling a pool, etc.). In any case, it is important to note that these outliers could affect the performance of the classification model.


#### 1. NaN processing

##### 1.1 Removing Data

Remove the data for an entire year of a consumer if there are more than `n` months missing.

In [ ]:
# Number of months missing for each year and consumer

nan_df = df.groupby(['Consumer_number','Year'])['Consumption'].apply(lambda x: x.isna().sum())
display(nan_df.unstack())

Consumers that exceed that threshold

In [ ]:
# Consumers that exceed that threshold

max_missing_months = 10
nan_counts = nan_df.reset_index()
to_remove = nan_counts[nan_counts['Consumption'] > max_missing_months]

display(to_remove)

Create and apply a mask to remove the consumers that exceed the threshold, now the size of the dataset has decreased to nearly 1000000 samples.

In [ ]:
# Remove rows with more than 10 missing values
mask = df.set_index(['Consumer_number', 'Year']).index.isin(to_remove.set_index(['Consumer_number', 'Year']).index)
filtered_data = df[~mask]

print("Size of the filtered_dataset: ", filtered_data.shape)

Finally, we can see that as we expect, there are NaN values for the months were the threshold is exceeded

In [ ]:
filtered_data.groupby(['Consumer_number','Year'])['Consumption'].apply(lambda x: x.isna().sum()).unstack()
df = filtered_data.copy()
display(df.head())

#### 3. Outliers removal

There are different available methods to remove outliers. Some of them are more robust than others or assume a normal distribution of the data. 

##### 3.1 Interquartile Range method. IQR
In this case, we can use the IQR method, which is an easy and robust method to remove outliers. The IQR is the difference between the 75th and 25th percentiles of the data. The outliers are defined as the values that are below the $\text{25th percentile} - k \cdot \text{IQR}$ or above the $\text{75th percentile} + k \cdot \text{IQR}$. The value of k is usually 1.5, but it can be adjusted to 3 depending on the data.

In [ ]:
p25 = df.loc[df['Consumption']!=0].groupby('Consumer_type')['Consumption'].quantile(0.25)
p75 = df.loc[df['Consumption']!=0].groupby('Consumer_type')['Consumption'].quantile(0.75)
iqr = p75 - p25
print('IQR: \n', iqr, sep='', end='\n\n')
k = 3
upper = p75 + k*iqr
# In this case, we are only interested in the upper limit for the outlier detection.
print('Upper limit: \n', upper, sep='', end='\n\n')
print('Number of outliers for each Consumer_type: \n', df.loc[df['Consumption']!=0].groupby('Consumer_type')['Consumption'].apply(lambda x: (x > upper[x.name]).sum()), sep='', end='\n\n')
print('Percentage of outliers for each Consumer_type: \n', df.loc[df['Consumption']!=0].groupby('Consumer_type')['Consumption'].apply(lambda x: (x > upper[x.name]).sum()/len(x)*100), sep='', end='\n\n')


df['OutlierIQR'] = df.apply(lambda x: 1 if x['Consumption'] > upper[x['Consumer_type']] else 0, axis=1)
print('Number of outliers: ', df.OutlierIQR.sum())

df_clean = df.loc[df['OutlierIQR'] == 0].copy()
df_clean.drop('OutlierIQR', axis=1, inplace=True)
df_clean.reset_index(drop=True, inplace=True)

print('Shape of the original DataFrame: ', df.shape)
print('Shape of the cleaned DataFrame: ', df_clean.shape)

# #--------------------------------- Apply the same process to the competition dataset ---------------------
# df_comp['OutlierIQR'] = df_comp.apply(lambda x: 1 if x['Consumption'] > upper[x['Consumer_type']] else 0, axis=1)
# df_clean_comp = df_comp.loc[df_comp['OutlierIQR'] == 0].copy()
# df_clean_comp.drop('OutlierIQR', axis=1, inplace=True)
# df_clean_comp.reset_index(drop=True, inplace=True)


##### 1.2 Imputing Data

For the users with a number of missing months below the threshold, we can impute them with the average value of:

- The mean consumption value of the consumer for a specific year
- The mean consumption value of the ``Consumer_type`` category wich the consumer belongs to (for a specific year)
- The monthly mean consumption value of the ``Consumer_type`` category wich the consumer belongs to (all the years)

**Note: As we are going to compute the mean values for each consumer and year, it is better to remove the outliers first.**

First we compute these means, and then we merge them with the original dataframe.

In [ ]:
# Compute the mean of the Consumption values for each Consumer_number and Year
consumer_yearly_avg = df_clean.groupby(['Consumer_number', 'Year'])['Consumption'].mean().reset_index(name='Consumer_Avg')

# Compute the mean of the Consumption values for each Consumer_type and Year
group_yearly_avg = df_clean.groupby(['Consumer_type', 'Year'])['Consumption'].mean().reset_index(name='Group_Avg')

# Compute the monthly mean of the Consumption values for each Consumer_number (all years)
group_monthly_avg = df_clean.groupby(['Consumer_type', 'Month'])['Consumption'].mean().reset_index(name='Group_Monthly_Avg')

# Merge con el DataFrame original
combined_data = pd.merge(df_clean, consumer_yearly_avg, on=['Consumer_number', 'Year'])
combined_data = pd.merge(combined_data, group_yearly_avg, on=['Consumer_type', 'Year'])
combined_data = pd.merge(combined_data, group_monthly_avg, on=['Consumer_type', 'Month'])

display(combined_data.head)

#--------------------------------- Apply the same process to the competition dataset ---------------------
# Compute the mean of the Consumption values for each Consumer_number and Year
consumer_yearly_avg_comp = df_comp.groupby(['Consumer_number', 'Year'])['Consumption'].mean().reset_index(name='Consumer_Avg')

# We don't have the Consumer_type in the competition dataset, so we can't compute the group averages for the Consumer_type
# # Compute the mean of the Consumption values for each Consumer_type and Year
# group_yearly_avg_comp = df_clean_comp.groupby(['Consumer_type', 'Year'])['Consumption'].mean().reset_index(name='Group_Avg')

# # Compute the monthly mean of the Consumption values for each Consumer_number (all years)
# group_monthly_avg_comp = df_clean_comp.groupby(['Consumer_type', 'Month'])['Consumption'].mean().reset_index(name='Group_Monthly_Avg')

# Merge con el DataFrame original
combined_data_comp = pd.merge(df_comp, consumer_yearly_avg_comp, on=['Consumer_number', 'Year'])
# combined_data_comp = pd.merge(combined_data_comp, group_yearly_avg_comp, on=['Consumer_type', 'Year'])
# combined_data_comp = pd.merge(combined_data_comp, group_monthly_avg_comp, on=['Consumer_type', 'Month'])

display(combined_data_comp.head)

Iterate over the rows of the dataframe and impute the NaN values with the corresponding mean values.

In [ ]:
# Substitution of NaN values with the mean of the group, month and consumer
combined_data['Consumption'] = combined_data.apply(
    lambda row: ((row['Consumer_Avg'] + row['Group_Avg'] + row['Group_Monthly_Avg']) / 3 
                 if pd.isna(row['Consumption']) else row['Consumption']), axis=1)


#--------------------------------- Apply the same process to the competition dataset ---------------------
combined_data_comp['Consumption'] = combined_data_comp.apply(
    lambda row: ((row['Consumer_Avg'])  
                 if pd.isna(row['Consumption']) else row['Consumption']), axis=1)

In [ ]:
display(combined_data.loc[combined_data.Consumer_number == "AABK96307399687530"].sort_values(by=['Year', 'Month']))
df = combined_data.copy()

#--------------------------------- Apply the same process to the competition dataset ---------------------
df_comp = combined_data_comp.copy()

In [ ]:
df.dropna(inplace=True)
df.isna().any()

df_comp.dropna(inplace=True)

In [ ]:
# Line graph of Compsumption vs Date for a specific Consumer_number "AABK96307399687530"
fig = px.line(df.loc[df['Consumer_number'] == "AABK96307399687530"], x='Date', y='Consumption', title='Consumption vs Date for a specific Consumer_number')
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Consumption [m3]',
)
fig.show()


#### Graphs after removing outliers

In [ ]:
fig = px.histogram(df.loc[df['Consumption']!=0], x='Consumption', nbins=50, color='Consumer_type', barmode='overlay', histnorm='percent', title='Consumption distribution for each Consumer_type')
fig.update_layout(
    xaxis_title='Consumption [m3]',
    yaxis_title='Percentage (by category) [%]',
)
fig.show()


fig = px.box(df.loc[(df['Consumption']!=0)], y='Consumption', color='Consumer_type', title='Consumption distribution for each Consumer_type')
fig.show()

fig = px.violin(df.loc[(df['Consumption']!=0)], y='Consumption', color='Consumer_type', title='Consumption distribution for each Consumer_type')
fig.show()

In [ ]:
# Save the cleaned DataFrame as a parquet file
df_clean = df.copy()
df_clean.to_parquet('df_clean.parquet.gzip', compression='gzip')

#--------------------------------- Apply the same process to the competition dataset ---------------------
df_comp_clean = df_comp.copy()
df_comp_clean.to_parquet('df_comp_clean.parquet.gzip', compression='gzip')

### Data Transformation

#### 0. Train/test split

First, we need to split the data into train and test sets. We will use the train set to train the model, and the test set to evaluate its performance. We will use a 80/20 split, which is a common practice. Also, this step should be done before any data transformation, in order to avoid data leakage, so the following steps parameters (encoding, scaling) should be trained only on the train set and then applied to the test set.

It is important to note that the split should be stratified, in order to preserve the same proportion of samples for each class in both sets (``straify=y`` parameter in the ``train_test_split`` function). 

Note: *train/test split is the easiest way to evaluate the performance of a model, but it may not be the best one. In this case, we have a time series dataset, so it would be better to use a time-based split, in order to avoid data leakage. For example, we could use the first 6 years for training and the last year for testing. Also, it may be better not to separate the data from the same consumer in the train and test sets. We have to fix this.*

#### 1. Feature encoding

##### 1.1 Cyclical features

The feature `Month` is cyclical. In order to take in account this dependance we can transform the features into two new ones, `sin` and `cos`, which will represent the circular nature of the data. This is called *cyclical encoding*.

Some drawbacks to have in mind:
- You are converting one information into two features.
- Decision trees based algorithms (Random Forest, Gradient Boosted Trees, XGBoost) build their split rules according to one feature at a time. This means that they will fail to process these two features simultaneously whereas the cos/sin values are expected to be considered as one single coordinates system.

##### 1.2 Categorical features

The features `Consumer_type` and `Installation_zone` are categorical. In order to use them in the classification task, we need to encode them. There are different methods to do this:

- One-hot encoding: creates a new column for each category, with a 1 if the sample belongs to that category, and 0 otherwise. This method is not recommended for high cardinality features (many categories), because it will create a lot of new features, which will increase the dimensionality of the dataset. In this case, we have 7 categories for `Consumer_type` and 49 for `Installation_zone`, so it is not recommended to use this method.
- Label encoding: assigns a number to each category. One disadvantage of this method is that it will assign a numerical order to the categories, which could be interpreted as a ranking. (it is not recommended to use this method).
- Frequency encoding/Target encoding: assigns a number to each category, based on the frequency of the category in the dataset. There are different methods to do this, like mean encoding, weight of evidence, etc.
- Hashing encoding: creates a new column for each category, with a 1 if the sample belongs to that category, and 0 otherwise. It is similar to one-hot encoding, but it uses a hashing function to reduce the number of features. It is useful for high cardinality features (many categories), because it will create less new features than one-hot encoding. 
- Leave one out encoding: assigns a number to each category, based on the frequency of the category in the dataset, but leaving out the current sample. It is similar to target encoding, but it is more robust to overfitting, as it applies some kind of regularization.


In [ ]:
# 0. Train/Test split

# Helper function for splitting the data into train and test sets

def split_by_id(df: pd.DataFrame, test_percent: float=0.2, random_state: int=123, plot: bool=True) -> (pd.DataFrame, pd.DataFrame):
    """
    Splits the dataframe into train and test, ensuring that no Consumer_number is present in both sets.

    This function splits the dataset into training and testing, with different consumers in each set, while
    maintaining the proportion of the target variable. It takes into account the Consumer_type variable, so that
    the proportion of each Consumer_type is the same in both sets (aprox).

    Parameters
    ----------
    df : pd.DataFrame
        The dataset to be split.
    test_percent : float, optional
        The percentage of the dataset to be used for testing. The default is 0.2.
    random_state : int, optional
        The random state to be used for reproducibility. The default is 123.
    plot : bool, optional
        Whether to plot the distribution of the Consumer_type in the train and test sets. The default is True.

    Returns
    -------
    pd.DataFrame
        The training set.
    pd.DataFrame
        The testing set.
    """
    id_by_type = df_clean.groupby('Consumer_type')['Consumer_number'].unique().to_dict()
    df_id_by_type = pd.DataFrame(list(id_by_type.items()), columns=['Consumer_type', 'Consumer_number'])
    df_id_by_type = df_id_by_type.explode('Consumer_number').reset_index(drop=True)
    df_id_by_type.head()

    # Split the Consumer_number into train and test while mantaining the proportion of the Consumer_type
    train_consumers, test_consumers = train_test_split(df_id_by_type, test_size=test_percent, random_state=random_state, stratify=df_id_by_type['Consumer_type'])

    train = df[df['Consumer_number'].isin(train_consumers['Consumer_number'])]
    test = df[df['Consumer_number'].isin(test_consumers['Consumer_number'])]


    if plot:
        # Plot the distribution of the Consumer_type in the train and test sets
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=train.drop_duplicates(subset=['Consumer_number'])['Consumer_type'], name='Train'))
        fig.add_trace(go.Histogram(x=test.drop_duplicates(subset=['Consumer_number'])['Consumer_type'],  name='Test'))
        fig.update_layout(
            title='Distribution of the Consumer_type in the train and test sets',
            xaxis_title='Consumer_type',
            yaxis_title='Number of Consumers',
            barmode='overlay',
        )
        # Apply styling to the plot to be more professional, paper-like
        # choose the figure font
        font_dict=dict(
                    size=12,
                    color='black'
                    )
        # general figure formatting
        fig.update_layout(font=font_dict,  # font formatting
                        plot_bgcolor='white',  # background color
                        width=850,  # figure width
                        height=700,  # figure height
                        margin=dict(r=10,t=50,b=10)  # remove white space 
                        )
        # x and y-axis formatting
        fig.update_yaxes(showline=True,  # add line at x=0
                        linecolor='black',  # line color
                        linewidth=2.4, # line size
                        ticks='outside',  # ticks outside axis
                        tickfont=font_dict, # tick label font
                        mirror='allticks',  # add ticks to top/right axes
                        tickwidth=2.4,  # tick width
                        tickcolor='black',  # tick color
                        )
        fig.update_xaxes(showline=True,
                        showticklabels=True,
                        linecolor='black',
                        linewidth=2.4,
                        ticks='outside',
                        tickfont=font_dict,
                        mirror='allticks',
                        tickwidth=2.4,
                        tickcolor='black',
                        )
        fig.show()

    return train, test


In [ ]:
# Restore de dataset from the parquet file
# df_clean = pd.read_parquet('df_clean.parquet.gzip')

train, test = split_by_id(df_clean, test_percent=0.2, random_state=123, plot=True)

target = 'Consumer_type'
features = ['Year', 'Month', 'Date', 'Installation_zone', 'Consumer_number', 'Consumption']

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

print('Train set shape: ', X_train.shape)
print('Test set shape: ', X_test.shape)

print('Train set target distribution: \n', y_train.value_counts(normalize=True), sep='', end='\n\n')
print('Test set target distribution: \n', y_test.value_counts(normalize=True), sep='', end='\n\n')

# 1. Baseline model

# Baseline model - always predict the most frequent class
y_pred = ['domestic']*len(y_test)
print('------ Baseline Model ------')
print('Accuracy score: ', accuracy_score(y_test, y_pred))
print('Precision score: ', precision_score(y_test, y_pred, average='macro', zero_division=0))
print('Recall score: ', recall_score(y_test, y_pred, average='macro'))
print('F1 score: ', f1_score(y_test, y_pred, average='macro'))



In [ ]:
# 1.1 Encoding the cyclical feature "Month"

# (df.Month-1) because we want to start from 0, not 1
X_train['Month_sin'] = np.sin((X_train.Month-1)*(2.*np.pi/12))
X_train['Month_cos'] = np.cos((X_train.Month-1)*(2.*np.pi/12))

X_test['Month_sin'] = np.sin((X_test.Month-1)*(2.*np.pi/12))
X_test['Month_cos'] = np.cos((X_test.Month-1)*(2.*np.pi/12))

# Parametric plot of the cyclical feature "Month"
fig = plt.figure(figsize=(6, 6))
plt.scatter(X_train.Month_sin, X_train.Month_cos)
plt.xlabel('Month_sin')
plt.ylabel('Month_cos')
plt.title('Parametric plot of the cyclical feature "Month"')
plt.show()


In [ ]:
# 1.2 Encoding the rest of the categorical features

# Encoding of the target variable. It will assign a number to each category.
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

print(le.classes_)
print(le.transform(le.classes_))

In [ ]:
# Encoding of the categorical features

# encoder = ce.LeaveOneOutEncoder(cols=['Installation_zone', 'Consumer_number'])
# encoder = ce.CatBoostEncoder(cols=['Installation_zone', 'Consumer_number'])
encoder = ce.TargetEncoder(cols=['Installation_zone'])
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

print(X_train.head(), '\n')
print(X_test.head())

print("Installation zone nunique values: ", X_train.Installation_zone.nunique())
print("Installation zone unique values: ", X_train.Installation_zone.unique())


#### 2. Feature scaling

Necessary for some algorithms that are sensitive to the scale of the features or distance based, like SVM, KNN, Neural Netwworks, etc.

##### 2.1 Standardization

Standardization is a common method to scale the features. It transforms the data to have a mean of 0 and a standard deviation of 1. It is useful for features that follow a normal distribution. Afected by outliers. Computed as:

$x_{scaled} = \frac{x - \mu}{\sigma}$

##### 2.2 Min-max scaling (Normalization)

Min-max scaling transforms the data values to the range 0 - 1. Afected by outliers. It is calculated as: 

$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$


##### 2.3 Robust scaling

Similar to Standardization but instead of divide by $\sigma$ (standard deviation), it uses the interquartile range (IQR) to scale the data. It is useful for features that have outliers, because it is more robust to them.

$x_{scaled} = \frac{x - \mu}{Q_3 - Q_1}$

##### 2.4 Power transformation

Apply a power transformation to the data, in order to make it more Gaussian-like. Then standardize it.



In [ ]:
scaler = RobustScaler()
# scaler = MinMaxScaler()
X_train['Consumption'] = scaler.fit_transform(np.array(X_train['Consumption']).reshape(-1, 1))
X_test['Consumption'] = scaler.transform(np.array(X_test['Consumption']).reshape(-1, 1))

print(X_train[:5], '\n')
print(X_test[:5])
print(X_train.describe())

In [ ]:
# Time series graph of consumption by consumer_type (scaled and without outliers)

data = pd.concat([X_train, X_test], axis=0)
data['Consumer_type'] = le.inverse_transform(pd.concat([pd.Series(y_train), pd.Series(y_test)], axis=0))

group = data.groupby(['Date','Consumer_type'])['Consumption'].mean().unstack()

fig = px.line(group, x=group.index, y=group.columns, title='Time series graph of consumption by consumer_type, scaled and without outliers')
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Consumption [m3]',
)
fig.show()


### Model Selection

In [ ]:
from imblearn.over_sampling import SMOTE
# 0. Train/Test split
# Splitting the data into train and test sets
target = 'Consumer_type'
features = ['Year', 'Month', 'Date', 'Installation_zone', 'Consumption']

# Sort dataframe by year
df_clean = df.sort_values('Year')

# Get the unique years and determine the split
unique_years = df_clean['Year'].unique()
train_years = unique_years[:3]
test_years = unique_years[-3:]

# Create train and test dataframes
train_df = df_clean[df_clean['Year'].isin(train_years)]
test_df = df_clean[df_clean['Year'].isin(test_years)]

X_train = train_df[features].copy()
y_train = train_df[target].copy()
X_test = test_df[features].copy()
y_test = test_df[target].copy()

print('Train set shape: ', X_train.shape)
print('Test set shape: ', X_test.shape)

print('Train set target distribution: \n', y_train.value_counts(normalize=True))
print('Test set target distribution: \n', y_test.value_counts(normalize=True))

# 1.1 Encoding the cyclical feature "Month"
# (df.Month-1) because we want to start from 0, not 1
X_train['Month_sin'] = np.sin((X_train.Month-1)*(2.*np.pi/12))
X_train['Month_cos'] = np.cos((X_train.Month-1)*(2.*np.pi/12))

X_test['Month_sin'] = np.sin((X_test.Month-1)*(2.*np.pi/12))
X_test['Month_cos'] = np.cos((X_test.Month-1)*(2.*np.pi/12))

# 1.2 Encoding the rest of the categorical features
# Encoding of the target variable. It will assign a number to each category.
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

print(le.classes_)
print(le.transform(le.classes_))

# Encoding of the categorical features
encoder = ce.TargetEncoder(cols=['Installation_zone', 'Date'])
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

print(X_train.head())
print(X_test.head())

scaler = RobustScaler()
X_train['Consumption'] = scaler.fit_transform(np.array(X_train['Consumption']).reshape(-1, 1))
X_test['Consumption'] = scaler.transform(np.array(X_test['Consumption']).reshape(-1, 1))

print(X_train[:5])
print(X_test[:5])
print(X_train.describe())

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print('Resampled train set shape: ', X_train_res.shape)
print('Resampled train set target distribution: \n', pd.Series(y_train_res).value_counts(normalize=True))

# 1. Baseline model
# Baseline model - always predict the most frequent class
y_pred = [np.argmax(np.bincount(y_train))]*len(y_test)
print('------ Baseline Model ------')
print('Accuracy score: ', accuracy_score(y_test, y_pred))
print('Precision score: ', precision_score(y_test, y_pred, average='macro', zero_division=0))
print('Recall score: ', recall_score(y_test, y_pred, average='macro'))
print('F1 score: ', f1_score(y_test, y_pred, average='macro'))

In [ ]:
# RFC model
from sklearn.ensemble import RandomForestClassifier
# train model
model_rfc = RandomForestClassifier(n_estimators=100)
model_rfc.fit(X_train, y_train)

# make predictions within own data
y_pred = model_rfc.predict(X_test)

# evaluate model within own data
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# PREDICTION ON competition.csv

df_competition = df_comp[features].copy()

# 1.1 Encoding the cyclical feature "Month"
# (df.Month-1) because we want to start from 0, not 1
df_competition['Month_sin'] = np.sin((df_competition.Month-1)*(2.*np.pi/12))
df_competition['Month_cos'] = np.cos((df_competition.Month-1)*(2.*np.pi/12))

# Encoding of the categorical features
df_competition = encoder.transform(df_competition)

df_competition['Consumption'] = scaler.transform(np.array(df_competition['Consumption']).reshape(-1, 1))




# use the trained models to make predictions on new data
pred_rfc = model_rfc.predict(df_competition)

# add the predictions to the competition DF
df_rfc = df_competition.copy()
df_rfc['Consumer_type'] = pred_rfc

# save the prediction DF to a new .csv file
df_rfc.to_csv('result_rfc.csv', index=False)

In [ ]:
final_df = df_rfc.copy()
final_df['Consumer_number'] = df_comp['Consumer_number']
final_df.groupby('Consumer_number')['Consumer_type'].apply(lambda x: statistics.mode(x))

final_df['Consumer_type'] = le.inverse_transform(final_df['Consumer_type'])

In [ ]:

from scipy import stats

final_df = df_rfc.copy()
final_df['Consumer_number'] = df_comp['Consumer_number']

# Función para encontrar el valor menos repetido o la moda si todos son igualmente comunes
def least_common_or_mode(serie):
    counts = serie.value_counts()
    # Si todos los valores son igualmente comunes, devuelve la moda
    if counts.min() == counts.max():
        try:
            # Devuelve la moda si es única
            return stats.mode(serie)[0][0]
        except:
            # Si hay múltiples modas, puedes decidir qué hacer, por ejemplo, devolver el primer valor
            return serie.iloc[0]
    # De lo contrario, devuelve el valor menos común
    return counts.idxmin()

# Aplicar la función para cada grupo
final_df['Consumer_type'] = final_df.groupby('Consumer_number')['Consumer_type'].apply(least_common_or_mode)

# Si necesitas invertir la transformación de etiquetas
# Asegúrate de que no haya NaNs antes de invertir la transformación de etiquetas
final_df.Consumer_type.isna().sum()
final_df['Consumer_type'] = le.inverse_transform(final_df['Consumer_type'].astype(int))

final_df


In [ ]:
final_df.Consumer_type.value_counts()

## Analysis

Old dataset analysis

In [ ]:
def scatter_graph(consumertype_df, consumertype):
  consumertype_df = df.loc[df['Consumer_type'] == consumertype]
  consumer_month = consumertype_df['Month'].astype(str) + '/' + consumertype_df['Year'].astype(str)
  consumer_water_consumed = [consumertype_df['Consumption']]

  plt.figure(figsize=(15,4))
  plt.scatter(consumer_month, consumer_water_consumed)

  plt.title("scatter plot " + consumertype)
  plt.xlabel("month/year")
  plt.ylabel("water consumed")
  plt.xticks(rotation=70, size=8)
  plt.show()

scatter_graph("domestic_df", "domestic")
scatter_graph("rural_domestic_df", "rural domestic")
scatter_graph("industrial_df", "industrial")
scatter_graph("rural_commercial_df", "rural commercial")
scatter_graph("construction_df", "construction")
scatter_graph("low_income_families_df", "low income families")
scatter_graph("rural_expansion_df", "rural expansion")

In [ ]:
df.loc[df['Consumer_type'] == consumertype]

In [ ]:
xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dec']
plt.figure(figsize=(15,4))

def monthly_avarage_consumption(consumertype_df, consumertype):
  consumertype_df = df.loc[df['Consumer_type'] == consumertype]
  avg_consumption = consumertype_df[['Month', 'Consumption']].groupby('Month').mean()
  plt.plot(xlabels, avg_consumption, label=consumertype)

monthly_avarage_consumption("domestic_df", "domestic")
monthly_avarage_consumption("rural_domestic_df", "rural domestic")
monthly_avarage_consumption("industrial_df", "industrial")
monthly_avarage_consumption("rural_commercial_df", "rural commercial")
monthly_avarage_consumption("construction_df", "construction")
monthly_avarage_consumption("low_income_families_df", "low income families")
monthly_avarage_consumption("rural_expansion_df", "rural expansion")

plt.xlabel("month", size=15)
plt.ylabel("avarege water consumed", size=15)
plt.xticks(size=15)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=5)

plt.show()

In [ ]:
xlabels = ('domestic', 'rural domestic', 'industrial', 'rural commercial', 'construction', 'low income families', 'rural expansion')
avarage_consumption = df[['Consumption', 'Consumer_type']].groupby('Consumer_type').mean()
plt.figure(figsize=(14,6))
plt.bar(xlabels, avarage_consumption.Consumption)


plt.show()

In [ ]:
nozero_df = df.loc[df['Consumption'] != 0]
xlabels = ('domestic', 'rural expansion', 'industrial', 'rural commercial', 'construction', 'low income families', 'rural expansion')

domestic_list = nozero_df.loc[nozero_df['Consumer_type'] == 'domestic'] ['Consumption'].tolist()
rural_domestic_list = nozero_df.loc[nozero_df['Consumer_type'] == 'rural domestic']['Consumption'].tolist()
industrial_list = nozero_df.loc[nozero_df['Consumer_type'] == 'rural domestic']['Consumption'].tolist()
rural_commercial_list = nozero_df.loc[nozero_df['Consumer_type'] == 'rural domestic']['Consumption'].tolist()
construction_list = nozero_df.loc[nozero_df['Consumer_type'] == 'rural domestic']['Consumption'].tolist()
low_income_families_list = nozero_df.loc[nozero_df['Consumer_type'] == 'rural domestic']['Consumption'].tolist()
rural_expansion_list = nozero_df.loc[nozero_df['Consumer_type'] == 'rural domestic']['Consumption'].tolist()

mode_consumption = [statistics.mode(domestic_list),
                    statistics.mode(rural_domestic_list),
                    statistics.mode(industrial_list),
                    statistics.mode(rural_commercial_list),
                    statistics.mode(construction_list),
                    statistics.mode(low_income_families_list),
                    statistics.mode(rural_expansion_list),
                    ]

median_consumption = [statistics.median(domestic_list),
                      statistics.median(rural_domestic_list),
                      statistics.median(industrial_list),
                      statistics.median(rural_commercial_list),
                      statistics.median(construction_list),
                      statistics.median(low_income_families_list),
                      statistics.median(rural_expansion_list),
                    ]

plt.bar(xlabels, mode_consumption)
plt.show()

plt.bar(xlabels, median_consumption)
plt.show